In [1]:
!pwd

/root/NER-project/code/training_pipeline/notebooks


In [61]:
from pathlib import Path
from datasets import load_dataset
import pandas as pd
import torch
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, pipeline
import evaluate
import numpy as np

from huggingface_hub import notebook_login
import wandb

In [3]:
DATA_DIR = Path("./data/wnut_17")
MODEL_DIR = Path("NER_hugg_model")

In [75]:
from datasets import load_dataset
wnut = load_dataset("wnut_17", cache_dir=DATA_DIR)

In [5]:
train_df = pd.DataFrame(wnut['train'])
train_df.tokens.apply(len).max()

41

In [6]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)

In [97]:
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words = True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

In [31]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [33]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

In [35]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [41]:
!pip install -q evaluate seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [42]:
import evaluate
seqeval = evaluate.load("seqeval")

In [47]:
labels = [label_list[i] for i in example[f"ner_tags"]]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-location',
 'I-location',
 'I-location',
 'O',
 'B-location',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [43]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
}

In [48]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
    }
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

In [63]:
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_DIR,
    num_labels = 13,
    id2label = id2label,
    label2id= label2id,
    torchscript=True
)

In [ ]:
# last_run_id = "NER_hugg_model"  # fetch the run_id from your wandb workspace

# # resume the wandb run from the run_id
# with wandb.init(
#     project=os.environ["WANDB_PROJECT"],
#     id=last_run_id,
#     resume="must",
# ) as run:
#     # Connect an Artifact to the run
#     my_checkpoint_name = f"checkpoint-{last_run_id}:latest"
#     my_checkpoint_artifact = run.use_artifact(my_model_name)

#     # Download checkpoint to a folder and return the path
#     checkpoint_dir = my_checkpoint_artifact.download()

#     # reinitialize your model and trainer
#     model = AutoModelForSequenceClassification.from_pretrained(
#         "<model_name>", num_labels=num_labels
#     )
#     # your awesome training arguments here.
#     training_args = TrainingArguments()

#     trainer = Trainer(model=model, args=training_args)

#     # make sure use the checkpoint dir to resume training from the checkpoint
#     trainer.train(resume_from_checkpoint=checkpoint_dir)

# project_name = "NER"  # Replace with your project name
# run_id = "your_run_id"  # Replace with specific run ID (if needed)

In [64]:
import os
os.environ["WANDB_PROJECT"]="NER"
os.environ["WANDB_LOG_MODEL"]="end"
os.environ["WANDB_NOTEBOOK_NAME"]="POC"

In [53]:
import wandb
wandb.login()

True

In [54]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "NER_hugg_model",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size=16,
    num_train_epochs = 2,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    report_to="wandb",
    logging_steps=1,
    push_to_hub=True
)

/root/anaconda3/envs/training-pipeline/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [65]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    train_dataset = tokenized_wnut["train"],
    eval_dataset = tokenized_wnut["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
    args = training_args
)

In [66]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.144000,0.345595,0.484780,0.398517,0.437436,0.944893
2,0.000600,0.347694,0.553729,0.405931,0.468449,0.947031


TrainOutput(global_step=426, training_loss=0.013838789077569127, metrics={'train_runtime': 43.118, 'train_samples_per_second': 157.428, 'train_steps_per_second': 9.88, 'total_flos': 91781128898820.0, 'train_loss': 0.013838789077569127, 'epoch': 2.0})

In [67]:
wandb.finish()

eval/accuracy,▁█
eval/f1,▁█
eval/loss,▁█
eval/precision,▁█
eval/recall,▁█
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▁▂▄▂▂▇▃▁▂▂▁▂▂▁▁▁▁▁▃▃▄▂▃▅▂▂▃▃▂▂▃▃█▂▅▄▄▃▄▃


In [172]:
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

# Masking one of the input tokens
masked_index = 8
# tokenized_text[masked_index] = "[MASK]"
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Creating a dummy input
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
dummy_input = [tokens_tensor, segments_tensors]

In [173]:
model = model.to('cpu')
traced_model = torch.jit.trace(model, dummy_input)
torch.jit.save(traced_model, "trace_model/traced_ner.pt")

/root/anaconda3/envs/training-pipeline/lib/python3.10/site-packages/transformers/modeling_utils.py:4565: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
/root/anaconda3/envs/training-pipeline/lib/python3.10/site-packages/transformers/models/distilbert/modeling_distilbert.py:230: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


In [174]:
loaded_model = torch.jit.load("trace_model/traced_ner.pt")
loaded_model.eval()

RecursiveScriptModule(
  original_name=DistilBertForTokenClassification
  (distilbert): RecursiveScriptModule(
    original_name=DistilBertModel
    (embeddings): RecursiveScriptModule(
      original_name=Embeddings
      (word_embeddings): RecursiveScriptModule(original_name=Embedding)
      (position_embeddings): RecursiveScriptModule(original_name=Embedding)
      (LayerNorm): RecursiveScriptModule(original_name=LayerNorm)
      (dropout): RecursiveScriptModule(original_name=Dropout)
    )
    (transformer): RecursiveScriptModule(
      original_name=Transformer
      (layer): RecursiveScriptModule(
        original_name=ModuleList
        (0): RecursiveScriptModule(
          original_name=TransformerBlock
          (attention): RecursiveScriptModule(
            original_name=MultiHeadSelfAttention
            (dropout): RecursiveScriptModule(original_name=Dropout)
            (q_lin): RecursiveScriptModule(original_name=Linear)
            (k_lin): RecursiveScriptModule(original

In [180]:
with torch.no_grad():
    pred = loaded_model(tokens_tensor, segments_tensors)
pred[0].shape


torch.Size([1, 14, 13])

In [181]:
label = np.argmax(pred, axis=3)
label[0][0]

array([ 0,  0,  0,  9, 10,  0,  0,  9, 10,  0,  0,  0,  0,  0])

In [182]:
pred_labels = [label_list[i] for i in label[0][0]]
pred_labels

['O',
 'O',
 'O',
 'B-person',
 'I-person',
 'O',
 'O',
 'B-person',
 'I-person',
 'O',
 'O',
 'O',
 'O',
 'O']